## Базовые требования к домашкам

1. Формат - jupyter-тетрадка или скрипт на питоне
2. Мы запускаем ваши тетрадки с нуля, поэтому следите, чтобы не было
- необъявленных переменных (удалили ячейку, а переменная продолжает использоваться)
- лишних принтов отладочной информации
3. Комментарии приветствуются!

## Задание
В рамках этого задания мы будем создавать программу, которая получая на вход отзыв, будет предсказывать, является отзыв положительным или отрицательным. Делать мы будем это таким образом: мы возьмём некоторое число заранее размеченных как положительные или отрицательные отзывов, выделим те слова, которые встречаются только в положительных или только в отрицательных отзывах, и будем считать, каких слов  в поступившем нам на проверку отзыве больше.


Мы будем работать по заранее определённому пайплайну:

1.  Сначала нам надо скачать дату -- соберите как минимум 60 (30 положительных  и 30 отрицательных) отзывов на похожие продукты (не надо мешать отзывы на отели с отзывами на ноутбуки) для составления "тонального словаря" (чем больше отзывов, тем лучше)  и 10 отзывов для проверки качества.   (2 балла в случае сбора путём парсинга, 1 - если найдете уже готовые данные или просто закопипастите без парсинга)

2. Токенизируйте слова,  приведите их к нижнему регистру и к начальной форме  (1 балл за токенизацию, 1 - за начальную форму)

3. Составьте 2 множества - в одном будут слова, которые встречаются только в положительных отзывах, а в другом - встречающиеся только в отрицательных. Попробуйте поиграть с частотностями и исключить шум (к примеру, выбросить слова, встречающиеся 1-2 раза) (2 балла) (если у вас получились пустые множества, уберите фильтр по частотности или увеличьте выборку)

4. Создайте функцию, которая будет определять, положительный ли отзыв или отрицательный в зависимости от того, какие слова встретились в нём, и посчитайте качество при помощи accuracy (1  - за коректно работающую функцию, 1 - за подсчёт accuracy)

5. Предложите как минимум 2 способа улучшить эту программу с помощью добавления к ней любых мулек (1 балл за описание словами, 2 - если реализуете хотя бы один способ)

В случае, если после долгих мучений в п. 3 множества по объективным причинам не получается (покажите, что пытались) - отправляйте жабу - зачтём полный балл

# Ваше задание принято, Олег

> Исполнитель назначен: и это **Андрей Якубой, группа БКЛ212**. Ждём аванс, Олег!

In [1]:
!pip3 install app_store_scraper pymystem3 nltk scikit-learn

In [2]:
from collections import Counter

import numpy as np
import pandas as pd

from pymystem3 import Mystem
from nltk.corpus import stopwords

from app_store_scraper import AppStore
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import accuracy_score

In [3]:
np.random.seed(42)

mystem = Mystem()
sw = set(stopwords.words("russian"))

## 1. Добываем отзывы

Где легко можно добыть порцию гневных комментариев о лагающем интерфейсе,
списавшихся за неподтверждённую подписку деньгах и обновлениях, которые
только всё портят? Правильно - в магазинах мобильных приложений!

Будем парсить отзывы из магазина приложений для яблочной техники. Рассмотрим
 две соц. сети: VK и OK. Вариант беспроигрышный: все два
 приложения невероятно популярны и имеют огромное количество как
 положительных, так и отрицательных отзывов. Кроме того, они достаточно
 коротки, чтобы наш наивный алгоритм хоть как-то сработал.

В App Store рейтинг приложению выставляется по пятибальной шкале. Чтобы
упростить задачу и свести классификацию к бинарной, мы будем брать в
качестве плохих только отзывы с рейтингом 1, а в качестве хороших - отзывы с
 рейтингом 5. Благо, приложения популярные, и мы можем себе позволить
 выбирать. А так как отзывы короткие, мы возьмём их много.

In [4]:
vk = AppStore(country="ru", app_name="VK")
ok = AppStore(country="ru", app_name="OK")

2023-09-21 12:42:10,300 [INFO] Base - Searching for app id
2023-09-21 12:42:12,030 [INFO] Base - Initialised: AppStore('ru', 'vk', 564177498)
2023-09-21 12:42:12,031 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/ru/app/vk/id564177498
2023-09-21 12:42:12,032 [INFO] Base - Searching for app id
2023-09-21 12:42:13,691 [INFO] Base - Initialised: AppStore('ru', 'ok', 398465290)
2023-09-21 12:42:13,692 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/ru/app/ok/id398465290


In [5]:
vk.review(how_many=1500, sleep=1)
ok.review(how_many=1500, sleep=1)

2023-09-21 12:42:19,276 [INFO] Base - [id:564177498] Fetched 80 reviews (80 fetched in total)
2023-09-21 12:42:26,310 [INFO] Base - [id:564177498] Fetched 180 reviews (180 fetched in total)
2023-09-21 12:42:33,242 [INFO] Base - [id:564177498] Fetched 280 reviews (280 fetched in total)
2023-09-21 12:42:40,976 [INFO] Base - [id:564177498] Fetched 380 reviews (380 fetched in total)
2023-09-21 12:42:48,656 [INFO] Base - [id:564177498] Fetched 480 reviews (480 fetched in total)
2023-09-21 12:42:55,848 [INFO] Base - [id:564177498] Fetched 580 reviews (580 fetched in total)
2023-09-21 12:43:03,478 [INFO] Base - [id:564177498] Fetched 680 reviews (680 fetched in total)
2023-09-21 12:43:10,668 [INFO] Base - [id:564177498] Fetched 780 reviews (780 fetched in total)
2023-09-21 12:43:17,913 [INFO] Base - [id:564177498] Fetched 880 reviews (880 fetched in total)
2023-09-21 12:43:25,282 [INFO] Base - [id:564177498] Fetched 980 reviews (980 fetched in total)
2023-09-21 12:43:32,534 [INFO] Base - [id:

In [6]:
vk.reviews[0:5]

[{'date': datetime.datetime(2019, 3, 27, 15, 58, 17),
  'developerResponse': {'id': 32873409,
   'body': 'Понимаем вас. Однако реклама позволяет платформе развиваться и сохранять множество бесплатных возможностей.\nЕсли вы видите некорректное объявление или столкнулись с проблемой, сообщите, пожалуйста, в Поддержку: vk.cc/si.',
   'modified': '2022-10-31T06:03:57Z'},
  'review': 'Вы не достоины даже 1 звезды, но мы люди, поставим 1. \n-1 звезда = из-за рекламы в музыке \n-2 звезда = из-за за платную подписку \n-3 звезда = за плохой дизайн \n-4 звезда = за лаги и баги, и реклама на видео',
  'rating': 5,
  'isEdited': False,
  'title': '1 звезду вы даже не достоины, но мы равнодушны',
  'userName': 'MakTavish•••••'},
 {'date': datetime.datetime(2019, 10, 1, 15, 22, 37),
  'developerResponse': {'id': 26694930,
   'body': 'Спасибо за Ваш конструктивный отзыв. Давайте разберёмся во всём вместе.\n\n1 и 2. Напишите, пожалуйста, в Поддержку о проблеме. Это можно сделать в приложении (кнопка «

In [7]:
ok.reviews[0:5]

[{'date': datetime.datetime(2018, 10, 24, 10, 23, 32),
  'developerResponse': {'id': 5646641,
   'body': 'Здравствуйте, спасибо за обращение, в последней версии приложения (7.29.1) проблема исправлена',
   'modified': '2018-10-24T14:20:32Z'},
  'review': 'Оперативно исправили ошибки в тексте.',
  'rating': 5,
  'isEdited': False,
  'userName': 'mrSega96',
  'title': 'Заголовок'},
 {'date': datetime.datetime(2018, 8, 31, 12, 52, 52),
  'review': 'Люблю одноклассники',
  'rating': 5,
  'isEdited': False,
  'userName': 'Victory0034',
  'title': 'Все нравится!'},
 {'date': datetime.datetime(2023, 1, 4, 4, 59, 43),
  'developerResponse': {'id': 34112921,
   'body': 'Здравствуйте! Нам приятно читать такие отзывы 😌 В нашем разделе «Увлечения» вы сможете смотреть видео, которые подходят для вас по интересам 🙂',
   'modified': '2023-01-05T14:56:01Z'},
  'review': 'Уже 15 лет общаюсь в российской социальной сети!',
  'rating': 5,
  'isEdited': False,
  'userName': 'Five Grand Adonyev',
  'title'

In [8]:
reviews = vk.reviews + ok.reviews
reviews = [(review["review"], review["rating"]) for review in reviews if
           review["rating"] == 1 or review["rating"] == 5]
reviews = pd.DataFrame(reviews, columns=["review_text", "is_positive"])
reviews["is_positive"] = reviews["is_positive"].replace({1: 0, 5: 1})

reviews.to_csv("app_store_reviews.csv", index=False)

reviews.head()

,review_text,is_positive
0,"Вы не достоины даже 1 звезды, но мы люди, пост...",1
1,"Думал хуже уже некуда, а не тут то было.\nМы п...",0
2,С каждым обновлением грузится все дольше и выл...,0
3,"Очень расстраивает, что когда это было Легенда...",0
4,Можно ещё больше рекламы в музыке?,0


In [9]:
len(reviews)

2340

Теперь у нас есть красивый датафрейм с бинарно классифицированными отзывами.
 Здесь цифрой 1 закодирован позитивный отзыв (который имел рейтинг 5), а
 цифрой 0 - негативный (который имел рейтинг 1).

У нас чуть больше 2000 отзывов, хоть и коротких - есть где разгуляться!
Однако сейчас их распределение неравномерное.

In [10]:
reviews["is_positive"].value_counts()

is_positive
0    1980
1     360
Name: count, dtype: int64

Люди очень злые. :с

Случайным образом выберем 300 положительных и 300 отрицательных отзывов
(максимально, сколько мы можем с учётом кол-ва положительных отзывов). 500
будем использовать для анализа, а 100 - для оценки.

In [11]:
positive_reviews = reviews[reviews["is_positive"] == 1]
negative_reviews = reviews[reviews["is_positive"] == 0]
positive_sample = positive_reviews.sample(n=250, random_state=42)
negative_sample = negative_reviews.sample(n=250, random_state=42)

reviews = reviews.drop(positive_sample.index)
reviews = reviews.drop(negative_sample.index)

eval_positive_reviews = reviews[reviews["is_positive"] == 1]
eval_negative_reviews = reviews[reviews["is_positive"] == 0]
eval_positive_sample = eval_positive_reviews.sample(n=50, random_state=42)
eval_negative_sample = eval_negative_reviews.sample(n=50, random_state=42)
eval_sample = pd.concat([eval_positive_sample, eval_negative_sample])


In [12]:
positive_sample["is_positive"].value_counts()

is_positive
1    250
Name: count, dtype: int64

In [13]:
negative_sample["is_positive"].value_counts()

is_positive
0    250
Name: count, dtype: int64

In [14]:
eval_sample["is_positive"].value_counts()

is_positive
1    50
0    50
Name: count, dtype: int64

## 2. Предобработка: токенизация и лемматизация

Для выполнения второго пункта нам хватит одной библиотеки - pymystem3,
которая является обёрткой вокруг легендарного Mystem. В библиотеку уже
встроен неплохой (для русского языка) токенизатор, который сразу срабатывает
 при лемматизации. Напишим функцию, которая предобрабатывает текст.

In [15]:
def preprocess_text(text):
    lemmas = mystem.lemmatize(text)
    cleaned_lemmas = [lemma.lower() for lemma in lemmas if lemma.isalpha()]
    return cleaned_lemmas

In [16]:
positive_sample["words"] = positive_sample["review_text"].apply(
    preprocess_text
)
negative_sample["words"] = negative_sample["review_text"].apply(
    preprocess_text
)
eval_sample["words"] = eval_sample["review_text"].apply(
    preprocess_text
)

In [17]:
positive_sample["words"].head()

1924    [выдавать, не, известный, ошибка, не, мочь, за...
1347    [исправлять, проблема, с, поиск, видео, молоде...
2091    [вернуть, функция, где, показывать, когда, зах...
2170    [крутой, соц, сеть, я, находить, себя, девушка...
1366    [простой, использование, и, легко, становиться...
Name: words, dtype: object

## 3. Выделяем уникальные слова

Сначало максимально наивно - без фильтрации - а потом в зависимости от того,
 что получится.

In [18]:
positive_words = set()
negative_words = set()

for words_list in positive_sample["words"]:
    positive_words.update(words_list)

for words_list in negative_sample["words"]:
    negative_words.update(words_list)

common_words = positive_words.intersection(negative_words)
positive_words -= common_words
negative_words -= common_words

In [19]:
len(positive_words)

354

In [20]:
len(negative_words)

1396

In [21]:
len(common_words)

534

In [22]:
positive_words

{'gggb',
 'good',
 'k',
 'ne',
 'odna',
 'ok',
 'pochemu',
 'pokazivayet',
 'top',
 'аба',
 'абу',
 'аватар',
 'аватарка',
 'авто',
 'адекватный',
 'азиаточка',
 'акаунт',
 'александр',
 'аллча',
 'американизировать',
 'бабушка',
 'баланс',
 'безобразие',
 'бессокирный',
 'бестолковый',
 'благодарить',
 'благодарный',
 'блестящий',
 'блок',
 'бонус',
 'борщ',
 'борьба',
 'бот',
 'бсд',
 'буквально',
 'вбадаб',
 'вверху',
 'вдруг',
 'включаться',
 'вмиг',
 'вовлвдмша',
 'возить',
 'возненавидеть',
 'возникать',
 'вой',
 'вол',
 'воообще',
 'впустую',
 'встреча',
 'встроеный',
 'выпускник',
 'вышеуказанный',
 'ггч',
 'готовить',
 'графон',
 'група',
 'гулнор',
 'дар',
 'дарить',
 'дата',
 'дебил',
 'деваться',
 'делить',
 'дизлайков',
 'дикпики',
 'дина',
 'дискриминация',
 'дороговатый',
 'дорогой',
 'доходить',
 'др',
 'друган',
 'е',
 'еденственный',
 'единый',
 'елена',
 'ж',
 'живой',
 'завершение',
 'заголовок',
 'заделывать',
 'залипать',
 'замечательно',
 'запрашивать',
 'запрос'

In [23]:
negative_words

{'переделывать',
 'минус',
 'господи',
 'отказываться',
 'принимать',
 'сон',
 'большой',
 'закон',
 'кончать',
 'ревнивый',
 'топовый',
 'обнищать',
 'обсуждение',
 'ливать',
 'крутить',
 'крайне',
 'использовать',
 'принятие',
 'введение',
 'унижение',
 'доступно',
 'практически',
 'расчленять',
 'закрытие',
 'интерфейс',
 'утрировать',
 'сдаваться',
 'промахиваться',
 'вместо',
 'комфорт',
 'недовольство',
 'везде',
 'шкурнадзор',
 'обсираться',
 'ерунда',
 'залетный',
 'умудряться',
 'детский',
 'во',
 'прагматичный',
 'процедура',
 'неприятно',
 'отстойный',
 'терпеть',
 'заминусовать',
 'двадцать',
 'курс',
 'instagram',
 'доставать',
 'блевотень',
 'снимать',
 'переманивать',
 'свинья',
 'впн',
 'обратный',
 'бессмысленный',
 'стоимость',
 'гетеросексуальный',
 'человеческий',
 'хопс',
 'оплата',
 'говнохип',
 'it',
 'тупить',
 'специально',
 'плей',
 'катиться',
 'хлам',
 'истеричка',
 'двоечник',
 'баг',
 'зато',
 'аудитория',
 'виза',
 'всплывать',
 'совершенно',
 'пора',
 'д

In [24]:
common_words

{'ios',
 'iphone',
 'se',
 'а',
 'автоматически',
 'админ',
 'администрация',
 'айфон',
 'аккаунт',
 'альбом',
 'андроид',
 'анкета',
 'аудиозапись',
 'бесить',
 'бесконечный',
 'бесплатно',
 'бесплатный',
 'благодаря',
 'блокировать',
 'больше',
 'больший',
 'большинство',
 'браузер',
 'бы',
 'быстро',
 'быть',
 'в',
 'вариант',
 'ваш',
 'вводить',
 'ведь',
 'вернуть',
 'версия',
 'весь',
 'вечер',
 'взламывать',
 'взять',
 'видео',
 'видеть',
 'видно',
 'виснуть',
 'вк',
 'вкладка',
 'включать',
 'внимание',
 'возможно',
 'возможность',
 'вообще',
 'восстанавливать',
 'время',
 'вроде',
 'все',
 'всегда',
 'вставлять',
 'входить',
 'вчера',
 'вчерашний',
 'вы',
 'выбирать',
 'выбор',
 'выглядеть',
 'выдавать',
 'вызывать',
 'выкидывать',
 'выкладывать',
 'вылет',
 'вылетать',
 'где',
 'главное',
 'главный',
 'глаз',
 'глючать',
 'говорить',
 'год',
 'голосовой',
 'гость',
 'готовый',
 'граница',
 'граф',
 'грузить',
 'грузиться',
 'группа',
 'да',
 'давать',
 'давно',
 'даже',
 'дале

Что-то совсем невразумительное и нерелевантное... Попробуем убрать те слова,
которые встречаются меньше трёх раз.

In [25]:
positive_counter = Counter()
negative_counter = Counter()

for words_list in positive_sample['words']:
    positive_counter.update(words_list)

for words_list in negative_sample['words']:
    negative_counter.update(words_list)

frequency_threshold = 3

positive_words = set()
negative_words = set()

for word, count in positive_counter.items():
    if count >= frequency_threshold:
        positive_words.add(word)

for word, count in negative_counter.items():
    if count >= frequency_threshold:
        negative_words.add(word)

common_words = positive_words.intersection(negative_words)
positive_words -= common_words
negative_words -= common_words

In [26]:
len(positive_words)

41

In [27]:
len(negative_words)

369

In [28]:
len(common_words)

173

In [29]:
common_words

{'а',
 'айфон',
 'блокировать',
 'больше',
 'бы',
 'быть',
 'в',
 'ваш',
 'вернуть',
 'версия',
 'весь',
 'видео',
 'вк',
 'вкладка',
 'возможность',
 'вообще',
 'время',
 'все',
 'вы',
 'вылетать',
 'где',
 'год',
 'группа',
 'да',
 'данный',
 'делать',
 'день',
 'для',
 'до',
 'добавлять',
 'добрый',
 'доступ',
 'друг',
 'другой',
 'если',
 'еще',
 'же',
 'за',
 'зайти',
 'заходить',
 'зачем',
 'звезда',
 'здравствовать',
 'знать',
 'и',
 'из',
 'или',
 'инстаграм',
 'интернет',
 'исправлять',
 'к',
 'каждый',
 'как',
 'какой',
 'клип',
 'кнопка',
 'когда',
 'количество',
 'который',
 'кто',
 'куда',
 'лента',
 'ли',
 'любой',
 'мир',
 'много',
 'можно',
 'мой',
 'момент',
 'мочь',
 'музыка',
 'мы',
 'на',
 'надоедать',
 'нажимать',
 'настройка',
 'находить',
 'начинать',
 'не',
 'нельзя',
 'нет',
 'ни',
 'но',
 'новый',
 'нормально',
 'нравиться',
 'ну',
 'нужно',
 'обновление',
 'обновлять',
 'один',
 'одноклассник',
 'ока',
 'он',
 'она',
 'они',
 'от',
 'отключать',
 'открываться

В принципе, слова получились довольно релевантные, но вот
несбалансированность ужасная! Наверное, раньше следовало догадаться, что
если позитивные отзывы в целом короче негативных, пороги частотности надо
брать разные. Будем записывать слова в позитивных отзывах, если они
встретились больше одного раза, а негативных - если больше 3 раз. А ещё
уберём стоп-слова.

In [30]:
positive_counter = Counter()
negative_counter = Counter()

for words_list in positive_sample['words']:
    positive_counter.update(words_list)

for words_list in negative_sample['words']:
    negative_counter.update(words_list)

positive_frequency_threshold = 2
negative_frequency_threshold = 4

positive_words = set()
negative_words = set()

for word, count in positive_counter.items():
    if count >= positive_frequency_threshold:
        positive_words.add(word)

for word, count in negative_counter.items():
    if count >= negative_frequency_threshold:
        negative_words.add(word)

common_words = positive_words.intersection(negative_words)
positive_words -= common_words
negative_words -= common_words
positive_words -= sw
negative_words -= sw

In [31]:
len(positive_words)

121

In [32]:
len(negative_words)

187

In [33]:
len(common_words)

211

In [34]:
positive_words

{'ios',
 'iphone',
 'аккаунт',
 'аудиозапись',
 'благодарить',
 'браузер',
 'бсд',
 'видно',
 'внимание',
 'восстанавливать',
 'вчера',
 'выглядеть',
 'выдавать',
 'выкидывать',
 'глючать',
 'гость',
 'грузить',
 'дата',
 'дело',
 'десять',
 'дина',
 'единственный',
 'желать',
 'запускать',
 'заставка',
 'зульфия',
 'игра',
 'имба',
 'инстаграм',
 'искать',
 'исчезать',
 'итог',
 'каталог',
 'категория',
 'класс',
 'комментарий',
 'круто',
 'крутой',
 'любить',
 'менять',
 'милый',
 'мобильный',
 'молвинский',
 'молодец',
 'надеяться',
 'назад',
 'наказание',
 'наш',
 'номер',
 'нравиться',
 'обложка',
 'обновляться',
 'обратно',
 'общение',
 'огромный',
 'открываться',
 'отлично',
 'отличный',
 'отображаться',
 'оценка',
 'перезагружать',
 'перезайти',
 'переустановить',
 'подружка',
 'подсказывать',
 'пожелание',
 'пожирать',
 'поздравлять',
 'поисковик',
 'получаться',
 'помнить',
 'помогать',
 'понравиться',
 'понятный',
 'почта',
 'правда',
 'правильно',
 'праздник',
 'предельно',

In [35]:
negative_words

{'apple',
 'tiktok',
 'vk',
 'абсолютно',
 'автоматически',
 'анкета',
 'аудитория',
 'бесплатно',
 'бесплатный',
 'бесполезный',
 'большой',
 'бред',
 'быстрый',
 'вводить',
 'везде',
 'взять',
 'вид',
 'видимо',
 'включать',
 'вконтакте',
 'вместо',
 'возможно',
 'всякий',
 'выбор',
 'вылезать',
 'выпускать',
 'г',
 'глаз',
 'говорить',
 'давно',
 'далеко',
 'данные',
 'держать',
 'дизайн',
 'днище',
 'дно',
 'должный',
 'доставать',
 'думать',
 'дурова',
 'жаль',
 'ждать',
 'желание',
 'женщина',
 'жизнь',
 'забывать',
 'загружать',
 'закрывать',
 'зато',
 'звонок',
 'знакомство',
 'идея',
 'изменение',
 'именно',
 'имя',
 'инстаграмма',
 'интересный',
 'интерфейс',
 'испортить',
 'история',
 'карусель',
 'контакт',
 'контент',
 'крайне',
 'купить',
 'куча',
 'лагать',
 'ладно',
 'либо',
 'мало',
 'меню',
 'мессенджер',
 'место',
 'месяц',
 'минута',
 'многие',
 'мужчина',
 'мусор',
 'навязчивый',
 'невозможно',
 'ненужный',
 'несколько',
 'неудобно',
 'неудобный',
 'низко',
 'никто

In [36]:
common_words

{'а',
 'айфон',
 'блокировать',
 'больше',
 'бы',
 'быть',
 'в',
 'ваш',
 'вернуть',
 'версия',
 'весь',
 'видео',
 'видеть',
 'вк',
 'вкладка',
 'возможность',
 'вообще',
 'время',
 'все',
 'вы',
 'вылетать',
 'где',
 'год',
 'голосовой',
 'группа',
 'да',
 'давать',
 'даже',
 'данный',
 'делать',
 'день',
 'деньги',
 'для',
 'до',
 'добавлять',
 'добрый',
 'доступ',
 'друг',
 'другой',
 'если',
 'еще',
 'же',
 'за',
 'зайти',
 'заходить',
 'зачем',
 'звезда',
 'звук',
 'здесь',
 'здравствовать',
 'знакомый',
 'знать',
 'и',
 'из',
 'или',
 'иногда',
 'интернет',
 'исправлять',
 'к',
 'каждый',
 'как',
 'какой',
 'клип',
 'кнопка',
 'когда',
 'количество',
 'который',
 'кто',
 'куда',
 'лента',
 'ли',
 'любой',
 'мир',
 'много',
 'можно',
 'мой',
 'момент',
 'мочь',
 'музыка',
 'мы',
 'на',
 'надо',
 'надоедать',
 'нажимать',
 'написать',
 'настройка',
 'находить',
 'начинать',
 'не',
 'неделя',
 'некоторый',
 'нельзя',
 'нет',
 'ни',
 'никакой',
 'ничто',
 'но',
 'новость',
 'новый',

Теперь супер!

## 4. Предсказываем отзывы

Пришло время сделать самое главное: написать наш наивный алгоритм и оценить
его работу. А чтобы всем казалось, что мы делаем что-то серьёзное, реализуем
 наш наивный алгоритм как модельку Sklearn! Да, как настоящие ML-модели!

In [37]:
class AppStoreClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, positive_words, negative_words):
        self.positive_words = positive_words
        self.negative_words = negative_words

    def fit(self, X, y=None):
        return self

    def predict(self, X):
        predictions = []
        for tokens in X:
            positive_count = sum(1 for token in tokens if token in positive_words)
            negative_count = sum(1 for token in tokens if token in negative_words)

            if positive_count > negative_count:
                predictions.append(1)
            else:
                predictions.append(0)
        return predictions

In [38]:
X = eval_sample["words"]
y = eval_sample["is_positive"]

clf = AppStoreClassifier(positive_words, negative_words)
y_pred = clf.predict(X)

In [39]:
accuracy_score(y, y_pred)

0.56

Вообще не очень. Но выборка у нас сбалансированная, так что результаты метрики
вполне объективные.

## 5. Пытаемся улучшить

Вариантов, как улучшить наш наивный алгоритм, довольно много. Сразу в голову
 приходит несколько идей.

Включить в словарь эмоджи. Кажется, для коротких отзывов в магазине
приложений это должно сработать, потому что эмодзи, в отличие от слов, более
 явно выражают положительные и отрицательные эмоции. Можно собрать списки
 эмоджи отдельно для положительных и отрицательных отзывов, и при
 предсказывании учитывать их вместе со словами.

При составлении словаря учитывать не только униграммы, но и биграммы (а,
может, и триграммы). Можно, как и с одиночными словами, добавить биграммы,
встречающиеся отдельно в положительных и отрицательных отзывах. Кажется, они
 должны нести в себе несколько больше смысла, чем отдельные слова, да и
 уникальность будет больше. Проблема, конечно, в том, что и вероятность
 встретить биграмму в другом наборе понижается, поэтому их можно
 рассматривать только как дополнение к словарю из отдельных слов, но не
 замену. Это, кстати, может решить проблему обработки отрицания.

Присваивать словам в словаре веса в зависимости от того, сколько раз каждое
из них встретилось в положительных и отрицательных отзывах, проранжировать
слова. Так, ранг у слова *супер* в списке слов из положительных отзывов явно
 будет выше, чем ранг слова *грузить*. Во время предсказания мы смотрим не
 только на то, слов из какого множества больше, но и на то, какого эти слова
  ранга, таким образом вычисляя вероятность того, что отзыв относится к
  положительным или отрицательным.

Наконец, в этих отзывах достаточно много опечаток - возможно, мы бы могли
просто скорректировать их с помощью какого-нибудь спеллчекера и улучшить
таким образом качество данных.

### Emoji: язык XXI века

> "I'm an emoji, and, even though i'm not exactly sure which one... I've gotta have some sort of purpose here, I know it!"

<figure>
<img src="https://static.wikia.nocookie.net/p__/images/7/78/Gene_emoji_movie.png/revision/latest?cb=20200613054406&path-prefix=protagonist" alt="Gene Meh" />
<figcaption align="center">Gene Meh</figcaption>
</figure>

Наши смайлики, к счастью, пока вроде бы знают свою цель, в отличие от Джина,
 и их можно попробовать использовать для более точной классификации отзывов.
  Создадим функцию, извлекающую эмоджи и текстовые смайлики из отзывов, а
  потом посмотрим на
  списки.

In [40]:
!pip3 install emot

In [41]:
from emot.core import emot

emot_obj = emot()

In [42]:
def extract_emoticons(text):
    emojis = emot_obj.emoji(text)["value"]
    text_emoticons = emot_obj.emoticons(text)["value"]
    return emojis + text_emoticons

In [43]:
positive_sample["emoticons"] = positive_sample["review_text"].apply(
    extract_emoticons
)
negative_sample["emoticons"] = negative_sample["review_text"].apply(
    extract_emoticons
)
eval_sample["emoticons"] = eval_sample["review_text"].apply(
    extract_emoticons
)

Выделяем уникальные для положительных и отрицательных отзывов эмотиконы

In [44]:
positive_emoticons = set()
negative_emoticons = set()

for emot_list in positive_sample["emoticons"]:
    positive_emoticons.update(emot_list)

for emot_list in negative_sample["emoticons"]:
    negative_emoticons.update(emot_list)

common_emoticons = positive_emoticons.intersection(negative_emoticons)
positive_emoticons -= common_emoticons
negative_emoticons -= common_emoticons

In [45]:
len(positive_emoticons)

32

In [46]:
len(negative_emoticons)

18

In [47]:
len(common_emoticons)

8

In [48]:
positive_emoticons

{':)',
 '=)',
 '☺',
 '✨',
 '❤',
 '🌟',
 '🌸',
 '🏳️\u200d🌈',
 '👄',
 '👍🏼',
 '👍🏽',
 '👹',
 '💋',
 '💕',
 '💪🏻',
 '🔥',
 '😅',
 '😉',
 '😊',
 '😍',
 '😎',
 '😏',
 '😓',
 '😳',
 '🙂',
 '🙏',
 '🤙',
 '🤝',
 '🤭',
 '🥰',
 '🥱',
 '🥳'}

In [49]:
negative_emoticons

{'D:',
 '❗',
 '👋',
 '👎',
 '👎🏻',
 '👎🏼',
 '🖕🏻',
 '😔',
 '😡',
 '😢',
 '😥',
 '🤢',
 '🤦🏻\u200d♀️',
 '🤦🏼\u200d♀️',
 '🤦🏼\u200d♂️',
 '🤩',
 '🤬',
 '🤮'}

In [50]:
common_emoticons

{'👌', '👍', '👍🏻', '👏', '😭', '🙏🏻', '🤪', '🥺'}

Здесь интересный момент: в отличие от списков слов, с эмотиконами
получилось, что позитивных намного больше, чем негативных. Выходит, в
положительных отзывах охотнее используют смайлики!

Теперь необходимо модифицировать наш классификатор. Поскольку список слов
никак не пересекается со списком эмотиконов, мы можем передавать в наш
классификатор слова и эмотиконы единым списком, нам нет необходимости
отделять одно от другого. Однако есть нюанс: эмотиконы, в отличие от слов,
выражают более-менее определённую эмоцию (если это не Джин), поэтому мы
увеличим их вес.

In [51]:
class AppStoreClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, positive_words, negative_words, positive_emoticons, negative_emoticons):
        self.positive_words = positive_words
        self.negative_words = negative_words
        self.positive_emoticons = positive_emoticons
        self.negative_emoticons = negative_emoticons

    def fit(self, X, y=None):
        return self

    def predict(self, X):
        predictions = []
        for tokens in X:
            positive_word_count = sum(1 for token in tokens if token in positive_words)
            negative_word_count = sum(1 for token in tokens if token in negative_words)
            positive_emoticon_count = sum(1 for token in tokens if token in self.positive_emoticons)
            negative_emoticon_count = sum(1 for token in tokens if token in self.negative_emoticons)
            positive_count = positive_word_count + (2 * positive_emoticon_count)
            negative_count = negative_word_count + (2 * negative_emoticon_count)
            if positive_count > negative_count:
                predictions.append(1)
            else:
                predictions.append(0)
        return predictions


In [52]:
X = eval_sample["words"] + eval_sample["emoticons"]
y = eval_sample["is_positive"]

clf = AppStoreClassifier(positive_words, negative_words, positive_emoticons, negative_emoticons)
y_pred = clf.predict(X)

In [53]:
accuracy_score(y, y_pred)

0.57

Всего на 1 процент! 😭

Тестовая выборка намного меньше, чем тренировочная. Похоже, смайликов в ней
 слишком мало, чтобы их анализ значительно повлиял на результат. Всё-таки
 отзывы в App Store слишком короткие, и для хорошей классификации нужна
 очень большая выборка (20000 и больше, я думаю). В домашних условиях, к
 сожалению, ограничения API App Store не позволяют взять так много, поэтому
 имеем, что имеем.

Но проверить смайлики в тестовой выборке мы можем.

In [54]:
positive_eval_emoticons = set()
negative_eval_emoticons = set()

for emot_list in eval_sample[eval_sample["is_positive"] == 1]["emoticons"]:
    positive_eval_emoticons.update(emot_list)

for emot_list in eval_sample[eval_sample["is_positive"] == 0]["emoticons"]:
    negative_eval_emoticons.update(emot_list)

In [55]:
positive_eval_emoticons

{'☺', '👍🏿', '👏', '😈', '😊'}

In [56]:
negative_eval_emoticons

{'👍🏻', '😡'}

Да, во-первых, эмотиконов мало, во-вторых, есть новые и те, которые по
нашему изначально составленному списку встречаются и там, и там. Но вот
зачем использовать 👍🏻 в негативном отзыве, я не понимаю... Надо думать,
сарказм и ирония теперь активно передаются и с помощью смайликов, и тут уже
нужны продвинутые методы анализа. Вероятно, вместо формирования уникального
списка эмоджи лучше может сработать заранее созданный датасет, в котором
смайлики уже разделены на позитивные и негативные, но пока неясно, где такой
 найти.

В любом случае, анализ эмотиконов, особенно в коротких отзывах, не стоит
сбрасывать со счетов: при достаточно большой выборке и наличие хорошо
классифицированного списка смайликов это может помочь определять тональность
 отзыва точнее.

# Ваш проект готов, Олег!

> ...а аванс мы так и не получили, Олег...
